# Ejercicios: Ecuaciones Lineales - A. Numerico Schwarz 1c 2019

## 1. Eliminación de Gauss y sustitución inversa

El método de eliminación de Gauss es un método directo muy efectivo que transforma la matriz de coeficientes original en una matriz triangular superior y luego aplica el método de sustitución inversa para obtener la solución del sistema dado.

Para ello se basa en la propiedad de que el sistema de ecuaciones no cambia al reemplazar filas por combinaciones lineales de las restantes

In [138]:
import numpy as np

print("Matriz ampliada: ")
A = np.matrix([['a11', 'a12', 'b1'], ['a21', 'a22', 'b2']])
A

Matriz ampliada: 


matrix([['a11', 'a12', 'b1'],
        ['a21', 'a22', 'b2']], dtype='<U3')

Queremos resolver el sistema $Ax = B$ tal que cada coeficiente de la matriz $A$ se multiplica por cada coeficiente de nuestro vector $x$.

1. Calculamos el coeficiente $m_{21} = \frac{a_{21}}{a_{11}}$

2. Luego calculamos los coeficientes: 
$$a^{*}_{22} = a_{22} - m_{21}a_{12}$$
$$a^{*}_{23} = a_{23} - m_{21}a_{13}$$
$$a^{*}_{24} = a_{24} - m_{21}a_{14}$$
$$b^{*}_2 = b_2 - m_{21}b_1$$


y así sucesivament para el resto de las filas, con lo que obtenemos la nueva matriz ampliada:


\begin{bmatrix} 
a_{11} & a_{12}     & a_{13}     & a_{14}     & | & b_1 \\
0      & a^{*}_{22} & a^{*}_{23} & a^{*}_{24} & | & b^{*}_2 \\
0      & a^{*}_{32} & a^{*}_{33} & a^{*}_{34} & | & b^{*}_3 \\
0      & a^{*}_{42} & a^{*}_{43} & a^{*}_{44} & | & b^{*}_4 \\
\end{bmatrix}


Repetimos sucesivamente, y luego comenzando por la última fila resolvemos una ecuación trivial, luego sustituimos en la ecuación anterior

In [139]:
def Gauss(A, B, M, ext=True):
    # Cantidad de veces a iterar para resolver el sistema en su totalidad dim(A)-1 veces.
    for l in range(A.shape[0]):

        # Cantidad de iteraciones para resolver por cada pivote. (columna a columna)
        for i in range(l, A.shape[0]):

            # Agregamos ceros en la fila.
            if(i == l):
                A[l+1:A.shape[0], i] = 0
                continue

            # Cantidad de iteraciones para resolver
            for j in range(i, A.shape[0]):
                A[i, j] = A.item(i, j) - M.item(i, l)*A.item(i-1, j)
            
            if(ext):
                B[0, i] = B.item(i) - M.item(i, l)*B.item(i-1)


In [140]:
A = np.matrix([[1, 2], 
               [3, 1]
              ])

B = np.matrix([ [4, 1] ])

M = np.matrix([ [0, 0],
               [(A.item(1,0) / A.item(0,0)), 0]
              ])


Gauss(A, B, M)
print(A)
print(B)

[[ 1  2]
 [ 0 -5]]
[[  4 -11]]


Una vez hallada la nueva matriz $A$, y la nueva matriz $B$ procedemos a realizar la sustitución de manera inversa iterativamente.

In [141]:
def sust_inv(A, B):
    x = [0, 0]
    ax = 0
    for i in range(A.shape[0]-1, -1, -1):
        for j in range(i, A.shape[0]):
            ax += A[i, j]*x[j]

        x[i] = (B[0, i] - ax)/A[i, i]
        ax = 0
    
    return x

In [142]:
x = sust_inv(A, B)

Para asegurarnos de que sea el resultado correcto, utilizamos la solución en ambos casos, con nuestra matriz $A$ original, y con nuestra matriz $A^{*}$ triangular inferior.

In [143]:
# Matrices luego de Gauss-Jordan
print(x*np.transpose(A)) 
print(B)

[[  4. -11.]]
[[  4 -11]]


In [144]:
# Matrices originales
A = np.matrix([[1, 2], 
               [3, 1]
              ])

B = np.matrix([ [4, 1] ])

print(x*np.transpose(A))
print(B)

[[4. 1.]]
[[4 1]]


Este procedimiento es muy útil puesto que se conoce exactamente la cantidad de pasos que edben efectuarse. Podemos conocer el costo computacional del método, establecer cuanto tiempo lleva todo el proceso.

Una vez analizado se puede ver que el costo de efectuar la eliminación de Gauss tiene un orden cubico $O(n³)$

## 2. Factorizacion LU

El método de eliminación de Gauss es un método muy potente. Sin embargo no siempre es conveniente su utilización.

Supongamos por un momento que para resolver un determinado problema debemos resolver el sistema de ecuaciones en forma anidada. Es decir, cada nueva solución depende del resultado obtenido en un paso anterior, o sea, cada vector $B$ depende de la solucion anterior: $B^{<i>} = f(x^{<i-1>})$

Un método muy eficiente para estos casos es la descomposición o factorización LU. Ésta consiste en descomponer la matriz A original en el producto de dos matrices:
    - Una triangular inferior: L
    - Una triangular superior: U

Con ellas creamos el siguiente sistema:
$Ax = LUx = B$ con $A = LU$
De esta forma obtenemos dos sistemas de ecuaciones:
$$ Ly = B $$
$$ Ux = y $$

Estas dos matrices corresponden a matrices anteriormente halladas, mediante un pequeño análisis podemos darnos cuenta que: 
$$U = A^{*}$$
$$L = M + I$$

Por lo tanto podemos utilizar los mismos métodos que utilizamos anteriormente para poder resolver la matriz:

In [163]:
A = np.matrix([[1, 2], 
               [8, 3]
              ])

B = np.matrix([ [2, 3] ])

M = np.matrix([ [0, 0],
               [(A.item(1,0) / A.item(0,0)), 0]
              ])

# Realizamos Gauss-Jordan en A, pero no en B.
Gauss(A, B, M, ext=False)
U = A
L = M + np.identity(M.shape[0])
print(L*U)

[[1. 2.]
 [8. 3.]]


En el primer caso para obtener la solución intermedia $y$, aplicamos la sustitución directa tal que:
$$ y_i = b_i - \sum_{j=1}^{i-1} l_{ij}y_j$$

In [164]:
def sust_dir(L, B):
    x = [0, 0] 
    ax = 0
    
    for i in range(L.shape[0]):
        for j in range(i):
            ax += L[i, j]*x[j]
        
        x[i] = B[0, i] - ax
    
    return x


In [165]:
y = sust_dir(L, B)
print(y)

[2, -13.0]


Una vez obtenido $y$, se aplica la sustitución inversa para obtener el vector $x$ solución del sistema.

In [166]:
y = np.matrix([y])
x = sust_inv(U, y)
print(x)

[0.0, 1.0]


Nuevamente nos fijamos si este valor resuelve el problema con las matrices originales.

In [168]:
A = np.matrix([[1, 2], 
               [8, 3]
              ])

B = np.matrix([ [2, 3] ])

print(x*np.transpose(A))
print(B)

[[2. 3.]]
[[2 3]]


Podemos ver que el costo computacional de este método, por utilizar la eliminación de Gauss, tendrá un costo similar al mismo. Sin embargo, en donde este método se destaca es en que no hay que repetir la triangulación de la matriz $A$ para distintos $B$.

## 3. Método de Cholesky

Si la matriz $A$ es simétrica definida positiva, entonces es posible obtener una matriz $S$ que cumpla:
$$ SS^{T} = A$$

Esta matriz se puede obtener a partir de la factorización $LU$.
La matriz simétrica A puede ser factorizada como $LDL^{T}$. Si además es definida positiva, entonces los coeficientes de D son positivos, por lo tanto podemos obtener $\sqrt(D)$, con lo cual tenemos:
$$ A = L\sqrt{D} \sqrt{D}L^{T} = SS^{T} $$

Las expresiones para obtener esta matriz $S$ son:

$$ s_{ii} = {[ a_{ii} - \sum_{k=1}^{i-1} s²_{ik} ]}^{1/2} $$
$$ s_{ji} = \frac{1}{s_{ii}} [ a_{ji} - \sum_{k=1}^{i-1} s_{jk}s_{ik} ] $$